In [17]:
import numpy             as np
import pandas            as pd
import matplotlib
import os
from   os      import getcwd
from   os      import listdir
from   os.path import isfile, join, isdir

import skimage
import tensorflow as tf

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.metrics import Accuracy



import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [8]:
def get_paths(foldNames):
  
    paths = dict.fromkeys(foldNames)

    for idx,g in enumerate(foldNames):
        fileNames = [f for f in listdir(join(trainPath,g)) if isfile(join(trainPath,g, f))]
        for i,f in enumerate(fileNames):
            fileNames[i] = join(trainPath,g,f)     
        paths[g] = fileNames
        
    return paths


# Setup

In [9]:
trainPath      = '../data/raw/train'
testPath       = '../data/raw/test_stg1'
rawdataPath    = '../data/raw'

fish_classes   = [f for f in listdir(trainPath) if isdir(join(trainPath, f))]
groupData      = pd.DataFrame ({'group': fish_classes})
fish_paths     = get_paths(fish_classes)

subsample_amnt  = 50  # len(files) = 3777
downsample_amnt = 2
ROWS            = 32 # int(720  / downsample_amnt)
COLS            = 32 # int(1280 / downsample_amnt)
CHANNELS        = 3
NUM_CATEGORIES  = len(fish_classes)

## `build_hdf5_image_dataset`

In [10]:
%%time

BUILD_HDF5_DATASET = False
IMAGE_SIZE         = 32
VALIDATION_SPLIT   = True
output_path        = join(rawdataPath, 'fish_dataset_{}x{}.h5'.format(IMAGE_SIZE, IMAGE_SIZE))
input_path         = join(rawdataPath, 'train')

if BUILD_HDF5_DATASET:
    # Build a HDF5 dataset (only required once)
    from tflearn.data_utils import build_hdf5_image_dataset


    build_hdf5_image_dataset(target_path        =input_path, 
                             image_shape        =(IMAGE_SIZE, IMAGE_SIZE), 
                             mode               ='folder', 
                             output_path        =output_path, 
                             categorical_labels =True, 
                             normalize          =True)

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 11 µs


In [11]:
%%time
from   sklearn.model_selection    import train_test_split

# Load HDF5 dataset
import h5py

h5f         = h5py.File(output_path, 'r')
X_all       = h5f['X'][()]
Y_all       = h5f['Y'][()]

# Split into 
if VALIDATION_SPLIT:
    X, X_valid, Y, Y_valid = train_test_split(X_all, Y_all, 
                                                          test_size    =0.2, 
                                                          random_state =23, 
                                                          stratify     =Y_all)


CPU times: user 14.8 ms, sys: 42.4 ms, total: 57.3 ms
Wall time: 56.1 ms


## `test model`

In [18]:
def dnn_test1():
    #needed to run this tensorflow operation in order to build the network and subsequently 
    #create the model, multiple times. Rebuilding without resetting the tf.Graph object produces
    #errors. Could also get around this issue by restarting kernel, but that's annoying.
    
    #python with statement: it will use this session for the code following the with statement 
    # and then automatically close the session
    with tf.Graph().as_default():
#         n_nodes_hl1 = 500
#         n_nodes_hl2 = 500
#         n_nodes_hl3 = 500
#         n_classes = 10
#         batch_size = 100
    
        # normalisation of images
        img_prep = ImagePreprocessing()
        img_prep.add_featurewise_zero_center()
        img_prep.add_featurewise_stdnorm()

        # Create extra synthetic training data by flipping & rotating images
        img_aug = ImageAugmentation()
        img_aug.add_random_flip_leftright()
        img_aug.add_random_rotation(max_angle=25.)

        
        #specific a specific device or gpu
#           with tf.device("/gpu:1"): 
        
        #input layer with shape of data specified. In this case, dimensions of our images, 
        #rows X cols X rgb array. The initial 'None' is for an unknown dimension reflecting the 
        #"number of samples that are processed in a batch"
        network = input_data(shape=[None, ROWS, COLS, 3],
                        data_preprocessing=img_prep,
                        data_augmentation=img_aug)
        
        # 1: Convolution layer with 32 filters, each 3x3x3
        conv_1 = conv_2d(network, 32, 3, activation='relu', name='conv_1')

        # 2: Max pooling layer
        network = max_pool_2d(conv_1, 2)

        # 3: Convolution layer with 64 filters
        conv_2 = conv_2d(network, 64, 3, activation='relu', name='conv_2')

        # 4: Convolution layer with 64 filters
        conv_3 = conv_2d(conv_2, 64, 3, activation='relu', name='conv_3')

        # 5: Max pooling layer
        network = max_pool_2d(conv_3, 2)

        # 6: Fully-connected 512 node layer
        network = fully_connected(network, 512, activation='relu')

        # 7: Dropout layer to combat overfitting
        network = dropout(network, 0.5)

        #output latyer
        network = fully_connected(network, 8, activation='softmax')
        
        # Configure how the network will be trained
        acc = Accuracy(name="Accuracy")
        
        network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0005, metric=acc)
        return tflearn.DNN(network)


In [20]:
# Define 

model_tf = dnn_test1()

# Start training (apply gradient descent algorithm). Will want to specify multiple epochs 
# typically unless just testing
model_tf.fit(X, Y, n_epoch=1,validation_set=(X_valid,Y_valid),
          show_metric=True, batch_size=15)

Training Step: 201  | total loss: 1.19422 | time: 4.924s
| Adam | epoch: 001 | loss: 1.19422 - Accuracy: 0.5567 -- iter: 3015/3021
Training Step: 202  | total loss: 1.18915 | time: 5.989s
| Adam | epoch: 001 | loss: 1.18915 - Accuracy: 0.5610 | val_loss: 1.11150 - val_acc: 0.6045 -- iter: 3021/3021
--


## `Train a model on the CIFAR-10 dataset and save it` 

In [ ]:
""" Convolutional network applied to CIFAR-10 dataset classification task.
References:
    Learning Multiple Layers of Features from Tiny Images, A. Krizhevsky, 2009.
Links:
    [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
"""


# Data loading and preprocessing
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

# Convolutional network building
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0,checkpoint_path='cifar10_cnn.tfl.ckpt')
model.fit(X, Y, n_epoch=100, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

In [ ]:
model.save("cifar10_cnn.tfl")

# `Refinement`

In [7]:
""" Finetuning Example. Using weights from model trained in
convnet_cifar10.py to retrain network for a new task (your own dataset).
All weights are restored except last layer (softmax) that will be retrained
to match the new task (finetuning).
"""

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

# Data loading
# Note: You input here any dataset you would like to finetune


# Redefinition of convnet_cifar10 network
network = input_data(shape=[None, ROWS, COLS, CHANNELS])
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = dropout(network, 0.75)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = dropout(network, 0.5)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
# Finetuning Softmax layer (Setting restore=False to not restore its weights)
softmax = fully_connected(network, NUM_CATEGORIES, activation='softmax', restore=False)
regression = regression(softmax, optimizer='adam',
                        loss='categorical_crossentropy',
                        learning_rate=0.001)

model = tflearn.DNN(regression, checkpoint_path='model_finetuning',
                    max_checkpoints=3, tensorboard_verbose=0)
# Load pre-existing model, restoring all weights, except softmax layer ones
model.load('cifar10_cnn.tfl')



ResourceExhaustedError: OOM when allocating tensor with shape[4096,512]
	 [[Node: FullyConnected/W/Adam_1/Assign = Assign[T=DT_FLOAT, _class=["loc:@FullyConnected/W"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](FullyConnected/W/Adam_1, Adam/zeros_14)]]

Caused by op u'FullyConnected/W/Adam_1/Assign', defined at:
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/stokesjd/anaconda/envs/tensorflow2

In [ ]:
# Start finetuning
model.fit(X, Y, n_epoch=1, shuffle=True,validation_set=(X_valid, Y_valid),
          show_metric=True, batch_size=4, snapshot_step=200,
          snapshot_epoch=False, run_id='model_finetuning')

model.save('model_finetuning')